In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

import xgboost as xgb
import catboost as cb

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Загрузка данных из CSV файла
data = pd.read_csv("bank-full.csv", delimiter=";")
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
# Разделение на признаки (X) и целевую переменную (y)
X = data.drop('y', axis=1)
y = data['y'].apply(lambda x: 1 if x=="yes" else 0)


# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение препроцессора для категориальных и числовых признаков
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
numerical_transformer = StandardScaler()

# Объединение препроцессоров
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

# Создание пайплайна для препроцессора
preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Применение препроцессора к данным
X_train = preprocessing_pipeline.fit_transform(X_train)
X_test = preprocessing_pipeline.transform(X_test)

In [7]:
from sklearn.svm import SVC

"""
Задание 2.3 (1 балл)
Сохраните в переменную model экземпляр класса SVC без параметров.
Пример:
var = ClassExample()
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

model = SVC()

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Определение параметров для GridSearchCV
"""
'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
Параметр 'kernel' определяет тип используемого ядра для построения гиперплоскости. Значение 'linear'
означает линейное ядро, которое строит гиперплоскость с помощью линейной функции. Значение 'poly' 
означает полиномиальное ядро, которое строит гиперплоскость с помощью полиномиальной функции. 
Значение 'rbf' означает радиальное базисное ядро (RBF), которое строит гиперплоскость с помощью 
радиальной базисной функции. Значение 'sigmoid' означает сигмоидное ядро, которое строит гиперплоскость 
с помощью сигмоидной функции.

'C': [0.1, 1.0, 10.0]
Параметр 'C' определяет параметр регуляризации модели SVM. Большее значение 'C' означает меньшую 
регуляризацию, что может привести к более точной модели на обучающих данных, но также может привести 
к переобучению. Меньшее значение 'C' означает большую регуляризацию, что может привести к менее точной 
модели, но более устойчивой к шуму и выбросам.

"""

params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1.0, 10.0]
}
# params = {
#     'kernel': ['linear'],
#     'C': [1.0]
# }

# Создание объекта GridSearchCV
"""
Задание 2.4 (1 балл)
Сохраните в переменную grid_search экземпляр класса GridSearchCV со следующими параметрами:
estimator=model
param_grid=params
scoring='f1_macro'
Пример:
var = GridSearchCV(par1=value1, par2=value2, par3=value3)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1_macro')

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Обучение модели на обучающих данных
grid_search.fit(X_train, y_train)

# Получение лучшей модели
best_model = grid_search.best_estimator_

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test)

# Получение лучших параметров SVC
SVC_best_params = grid_search.best_params_
print("SVC_best_params: ", SVC_best_params)

# Вычисление F1-меры
SVC_f1 = f1_score(y_test, y_pred, average='macro')
print("SVC_f1: ", SVC_f1)

SVC_best_params:  {'C': 10.0, 'kernel': 'rbf'}
SVC_f1:  0.7416478867533149


In [4]:
from sklearn.ensemble import GradientBoostingClassifier

"""
Задание 2.9 (1 балл)
Сохраните в переменную model экземпляр класса GradientBoostingClassifier без параметров.
Пример:
var = ClassExample()
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

model = GradientBoostingClassifier()

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Определение параметров для GridSearchCV
"""
'n_estimators': [100, 200, 300]
Параметр 'n_estimators' определяет количество деревьев в градиентном бустинге. Большее количество деревьев 
может повысить точность модели, но может также увеличить время обучения и инференса.

'max_depth': [3, 5, 10]
Параметр 'max_depth' определяет максимальную глубину дерева решений в градиентном бустинге. Значение 3, 5 
или 10 указывает конкретное ограничение на глубину дерева.

'learning_rate': [0.1, 0.01, 0.001]
Параметр 'learning_rate' определяет скорость обучения модели. Меньшие значения 'learning_rate' могут 
требовать большего количества деревьев для достижения высокой точности модели.

"""

params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.1, 0.01, 0.001]
}

# params = {
#     'n_estimators': [30, 50, 100],
#     'max_depth': [5],
#     'learning_rate': [0.1, 0.01]
# }

# Создание объекта GridSearchCV
"""
Задание 2.10 (1 балл)
Сохраните в переменную grid_search экземпляр класса GridSearchCV со следующими параметрами:
estimator=model
param_grid=params
scoring='f1_macro'
Пример:
var = GridSearchCV(par1=value1, par2=value2, par3=value3)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1_macro')

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Обучение модели на обучающих данных
grid_search.fit(X_train, y_train)

# Получение лучшей модели
best_model = grid_search.best_estimator_

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test)

# Получение лучших параметров GradientBoostingClassifier
GradientBoostingClassifier_best_params = grid_search.best_params_
print("GradientBoostingClassifier_best_params: ", GradientBoostingClassifier_best_params)

# Вычисление F1-меры
GradientBoostingClassifier_f1 = f1_score(y_test, y_pred, average='macro')
print("GradientBoostingClassifier_f1: ", GradientBoostingClassifier_f1)

GradientBoostingClassifier_best_params:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
GradientBoostingClassifier_f1:  0.7574285600990863


In [5]:
import xgboost as xgb

"""
Задание 2.11 (1 балл)
Сохраните в переменную model экземпляр класса xgb.XGBClassifier без параметров.
Пример:
var = ClassExample()
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

model = xgb.XGBClassifier()

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Определение параметров для GridSearchCV
"""
'n_estimators': [100, 200, 300]
Параметр 'n_estimators' определяет количество деревьев в ансамбле. Большее количество деревьев может 
улучшить качество модели, но может также увеличить время обучения и инференса.

'max_depth': [3, 5, 10]
Параметр 'max_depth' определяет максимальную глубину дерева. Увеличение глубины дерева может привести к 
более сложной модели, которая может переобучиться на тренировочных данных.

'learning_rate': [0.1, 0.01, 0.001]
Параметр 'learning_rate' определяет скорость обучения модели. Меньшие значения 'learning_rate' требуют 
большего числа деревьев для достижения хорошей точности модели.

"""

params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.1, 0.01, 0.001]
}
# params = {
#     'n_estimators': [30, 50, 100],
#     'max_depth': [3, 5],
#     'learning_rate': [0.1, 0.01]
# }

# Создание объекта GridSearchCV
"""
Задание 2.12 (1 балл)
Сохраните в переменную grid_search экземпляр класса GridSearchCV со следующими параметрами:
estimator=model
param_grid=params
scoring='f1_macro'
Пример:
var = GridSearchCV(par1=value1, par2=value2, par3=value3)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1_macro')

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Обучение модели на обучающих данных
grid_search.fit(X_train, y_train)

# Получение лучшей модели
best_model = grid_search.best_estimator_

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test)

# Получение лучших параметров XGBClassifier
XGBClassifier_best_params = grid_search.best_params_
print("XGBClassifier_best_params: ", XGBClassifier_best_params)

# Вычисление F1-меры
XGBClassifier_f1 = f1_score(y_test, y_pred, average='macro')
print("XGBClassifier_f1: ", XGBClassifier_f1)

XGBClassifier_best_params:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
XGBClassifier_f1:  0.7595293560667413


In [6]:
import catboost as cb

"""
Задание 2.13 (1 балл)
Сохраните в переменную model экземпляр класса cb.CatBoostClassifier с параметром silent=True.
Пример:
var = ClassExample(silent=True)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

model = cb.CatBoostClassifier(silent=True)

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Определение параметров для GridSearchCV
"""
'n_estimators': [100, 200, 300]
Параметр 'n_estimators' определяет количество деревьев в ансамбле. Большее количество деревьев может 
улучшить качество модели, но может также увеличить время обучения и инференса.

'max_depth': [3, 5, 10]
Параметр 'max_depth' определяет максимальную глубину дерева. Увеличение глубины дерева может привести к 
более сложной модели, которая может переобучиться на тренировочных данных.

'learning_rate': [0.1, 0.01, 0.001]
Параметр 'learning_rate' определяет скорость обучения модели. Меньшие значения 'learning_rate' требуют 
большего числа деревьев для достижения хорошей точности модели.

"""

params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.1, 0.01, 0.001]
}
# params = {
#     'n_estimators': [30, 50, 100],
#     'max_depth': [3, 5],
#     'learning_rate': [0.1, 0.01]
# }

# Создание объекта GridSearchCV
"""
Задание 2.14 (1 балл)
Сохраните в переменную grid_search экземпляр класса GridSearchCV со следующими параметрами:
estimator=model
param_grid=params
scoring='f1_macro'
Пример:
var = GridSearchCV(par1=value1, par2=value2, par3=value3)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1_macro')

# raise NotImplementedError()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

# Обучение модели на обучающих данных
grid_search.fit(X_train, y_train)

# Получение лучшей модели
best_model = grid_search.best_estimator_

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test)

# Получение лучших параметров CatBoostClassifier
CatBoostClassifier_best_params = grid_search.best_params_
print("CatBoostClassifier_best_params: ", CatBoostClassifier_best_params)

# Вычисление F1-меры
CatBoostClassifier_f1 = f1_score(y_test, y_pred, average='macro')
print("CatBoostClassifier_f1: ", CatBoostClassifier_f1)

CatBoostClassifier_best_params:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
CatBoostClassifier_f1:  0.7560220013076322


In [8]:
# Вывод результатов моделей классификации 
data = [ 
    ['SVC', f'{SVC_f1}', f'{SVC_best_params}'],
    ['GradientBoostingClassifier', f'{GradientBoostingClassifier_f1}', f'{GradientBoostingClassifier_best_params}'],
    ['XGBClassifier', f'{XGBClassifier_f1}', f'{XGBClassifier_best_params}'],
    ['CatBoostClassifier', f'{CatBoostClassifier_f1}', f'{CatBoostClassifier_best_params}']
]


# Создание DataFrame из данных
df = pd.DataFrame(data, columns=['Модель', 'F1', 'Параметры'], index=range(1, 1 + len(data)))
pd.set_option('display.max_colwidth', None)
# Отображение DataFrame
df.head(len(data))

,Модель,F1,Параметры
1,SVC,0.7416478867533149,"{'C': 10.0, 'kernel': 'rbf'}"
2,GradientBoostingClassifier,0.7574285600990863,"{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}"
3,XGBClassifier,0.7595293560667413,"{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}"
4,CatBoostClassifier,0.7560220013076322,"{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}"
